In [3]:
import pandas as pd

In [5]:
import sqlite3

In [7]:
orders = pd.read_csv("orders.csv")

In [9]:
users = pd.read_json("users.json")

In [11]:
conn = sqlite3.connect(":memory:")

In [13]:
cursor = conn.cursor()


In [15]:
with open("restaurants.sql", "r") as f:
    sql_script = f.read()

In [17]:
cursor.executescript(sql_script)

In [19]:
restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)

In [21]:
merged_1 = orders.merge(users, on="user_id", how="left")

In [23]:
final_df = merged_1.merge(restaurants, on="restaurant_id", how="left")

In [25]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

In [27]:
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [29]:
final_df.info


<bound method DataFrame.info of       order_id  user_id  restaurant_id  order_date  total_amount  \
0            1     2508            450  18-02-2023        842.97   
1            2     2693            309  18-01-2023        546.68   
2            3     2084            107  15-07-2023        163.93   
3            4      319            224  04-10-2023       1155.97   
4            5     1064            293  25-12-2023       1321.91   
...        ...      ...            ...         ...           ...   
9995      9996     2528            249  21-05-2023       1211.96   
9996      9997     2867            267  06-08-2023       1188.05   
9997      9998      522            420  11-11-2023        979.44   
9998      9999      319            492  08-09-2023       1105.93   
9999     10000      457            439  21-10-2023        879.58   

                   restaurant_name_x       name       city membership  \
0                  New Foods Chinese  User_2508  Hyderabad    Regular   
1    

In [35]:
gold_city_revenue = final_df[final_df["membership"] == "Gold"] \
    .groupby("city")["total_amount"].sum().sort_values(ascending=False)

gold_city_revenue


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [39]:
cuisine_avg = final_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)

cuisine_avg


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [41]:
user_total = final_df.groupby("user_id")["total_amount"].sum()

users_over_1000 = (user_total > 1000).sum()
users_over_1000


2544

In [43]:
import pandas as pd


bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0 – 3.5", "3.6 – 4.0", "4.1 – 4.5", "4.6 – 5.0"]

final_df["rating_range"] = pd.cut(final_df["rating"], bins=bins, labels=labels, include_lowest=True)


rating_revenue = final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)
rating_revenue


C:\Users\Sakshi\AppData\Local\Temp\ipykernel_5412\2368480642.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rating_revenue = final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


rating_range
4.6 – 5.0    2197030.75
3.0 – 3.5    2136772.70
4.1 – 4.5    1960326.26
3.6 – 4.0    1717494.41
Name: total_amount, dtype: float64

In [45]:
gold_avg_city = final_df[final_df["membership"] == "Gold"] \
    .groupby("city")["total_amount"].mean().sort_values(ascending=False)

gold_avg_city


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [47]:
rest_per_cuisine = final_df.groupby("cuisine")["restaurant_id"].nunique()

rev_per_cuisine = final_df.groupby("cuisine")["total_amount"].sum()

rest_per_cuisine, rev_per_cuisine


(cuisine
 Chinese    120
 Indian     126
 Italian    126
 Mexican    128
 Name: restaurant_id, dtype: int64,
 cuisine
 Chinese    1930504.65
 Indian     1971412.58
 Italian    2024203.80
 Mexican    2085503.09
 Name: total_amount, dtype: float64)

In [49]:
total_orders = len(final_df)
gold_orders = len(final_df[final_df["membership"] == "Gold"])

percentage_gold = (gold_orders / total_orders) * 100
percentage_gold



49.87

In [53]:
order_count = final_df.groupby("restaurant_name_x")["order_id"].count()

avg_value = final_df.groupby("restaurant_name_x")["total_amount"].mean()

filtered = avg_value[order_count < 20]

filtered.sort_values(ascending=False).head(10)


restaurant_name_x
Hotel Dhaba Multicuisine                 1040.222308
Sri Mess Punjabi                         1029.180833
Ruchi Biryani Punjabi                    1002.140625
Sri Delights Pure Veg                     989.467222
Classic Kitchen Family Restaurant         973.167895
Hotel Dhaba Chinese                       973.125556
Amma Mess Pure Veg                        965.299444
Hotel Biryani Pure Veg                    964.577692
Annapurna Curry House Multicuisine        954.512353
Taste of Restaurant Family Restaurant     948.220000
Name: total_amount, dtype: float64

In [55]:
avg_value["Ruchi Mess Multicuisine"] if "Ruchi Mess Multicuisine" in avg_value else "Not Found"


851.22625

In [57]:
combos = final_df.groupby(["membership", "cuisine"])["total_amount"].sum()
combos


membership  cuisine
Gold        Chinese     977713.74
            Indian      979312.31
            Italian    1005779.05
            Mexican    1012559.79
Regular     Chinese     952790.91
            Indian      992100.27
            Italian    1018424.75
            Mexican    1072943.30
Name: total_amount, dtype: float64

In [59]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])

final_df["quarter"] = final_df["order_date"].dt.quarter

quarter_rev = final_df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)
quarter_rev


C:\Users\Sakshi\AppData\Local\Temp\ipykernel_5412\1849616899.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(final_df["order_date"])


quarter
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64

In [61]:
gold_orders_count = len(final_df[final_df["membership"] == "Gold"])
gold_orders_count


4987

In [63]:
hyd_revenue = final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
round(hyd_revenue)


1889367

In [65]:
distinct_users = final_df["user_id"].nunique()
distinct_users


2883

In [67]:
gold_avg = final_df[final_df["membership"] == "Gold"]["total_amount"].mean()
round(gold_avg, 2)


797.15

In [69]:
orders_high_rating = len(final_df[final_df["rating"] >= 4.5])
orders_high_rating


3374

In [71]:
gold_chennai_orders = len(final_df[(final_df["membership"] == "Gold") & (final_df["city"] == "Chennai")])
gold_chennai_orders


1337